# Credit

This notebook is inspired from the repository [Python data structures: Trees](https://github.com/LinkedInLearning/python-data-structures-trees-2834010) by [Ryan Mitchell](https://github.com/REMitchell).  
Watching actively the real course on LinkedIn Learning is highly recommended.  
These are simply my notes and exercises.

# Creating our first nodes

Node with value `10` is the root node.  
We attach `5` to left and `15` to the right.  
We attach `2` to the left of `5` and `6` to the right of `5`.  
We attach `13` to the left of `15` and `10000` to the right of `15`.  
The tree is printed for verification.  


In [2]:
from tree import Node
from tree import Tree

node = Node(10)

node.left = Node(5)
node.right = Node(15)

node.left.left = Node(2)
node.left.right = Node(6)

node.right.left = Node(13)
node.right.right = Node(10000)

# from here on, we use the print functionality from the complete class  
# so we verify that the tree looks like expected
tree = Tree(node, "The whole tree")
tree.print()

fifteen = Tree(node.right, "Subtree: Fifteen")
fifteen.print()

five = Tree(node.left, "Subtree: Five")
five.print()

The whole tree 
      10  
   /     \
  5       15      
 / \     / \
2   6   13  10000

Subtree: Fifteen 
  15  
 / \
13  10000

Subtree: Five 
  5   
 / \
2   6   



# Search

We build a tree and try to find a specific entry.  
The code is replicated in this notebook for reference (local_function).  
The reference function is also used as comparison.  
We also try to look for a non existing entry, using local_function.  

In [3]:
from tree import Node
from tree import Tree

# search is implemented using recursion
#
# return the node when data == target, or
#   search left if left exists and current data is > target, or
#   search right if right exists and current data is < target, or
#   return None

def search_node(node, target):
    if node.data == target:
        # found!
        return node
    if node.left and node.data > target:
        return search_node(node.left, target)
    if node.right and node.data < target:
        return search_node(node.right, target)
    return None

def print_search_result(node, target, method):
    result = search_node(node, target)
    print("Found {} (using {})".format(result.data, method) if result 
        else "Target {} not Found (using {})".format(target, method))

node = Node(10)

node.left = Node(5)
node.right = Node(15)

node.left.left = Node(2)
node.left.right = Node(6)

node.right.left = Node(13)
node.right.right = Node(10000)

myTree = Tree(node, "Sample tree, does it contain my entry?")
myTree.print()

# Using the function in the complete class:
print("Using full library")
found10k = node.search(10000)
print("Found {} (using reference library)".format(found10k.data) if found10k else "Not Found (using reference library")

# search existing value with local function
print_search_result(node, 10000, "local_function")

# try to look for non-existing value with local function
print_search_result(node, 54, "local_function")


Sample tree, does it contain my entry? 
      10  
   /     \
  5       15      
 / \     / \
2   6   13  10000

Using full library
Found it!
Found 10000 (using reference library)
Found 10000 (using local_function)
Target 54 not Found (using local_function)


# Traverse

Traverse inOrder, preOrder, postOrder.  
We use a TraverseCollector class to collect the encountered entries.  
A TraversalExecutor class offers a static "execute" method which accepts the traverse function and a label.  
All the results are printed out in individual rows.

In [8]:
from tree import Node
from tree import Tree

class TraverseCollector():
    
    def __init__(self):
        self._list = []
        
    def collect(self, data):
        self._list.append(data)
        
    def getList(self):
        return self._list.copy()

class TraversalExecutor():
    
    def __init__(self, traverseFunction, collector):
        self._traverseFunction = traverseFunction
        self._collector = collector
        
    def _traverse(self, node, label):
        # traverse the node collecting data
        self._traverseFunction(node, self._collector)
        # create the list of collected entries
        str_list = [str(x) for x in self._collector.getList()]
        # create a row with values separated by commas
        output = ", ".join(str_list)
        # finally print
        print("{}: {}".format(label, output))
        
    # unique entrypoint
    def execute(node, traverseFunction, label):
        executor = TraversalExecutor(traverseFunction, TraverseCollector())
        executor._traverse(node, label)
        
def traverseInOrder(node, collector):
    if node.left:
        traverseInOrder(node.left, collector)
    collector.collect(node.data)
    if node.right:
        traverseInOrder(node.right, collector)

def traversePreOrder(node, collector):
    collector.collect(node.data)
    if node.left:
        traversePreOrder(node.left, collector)
    if node.right:
        traversePreOrder(node.right, collector)

def traversePostOrder(node, collector):
    if node.left:
        traversePostOrder(node.left, collector)
    if node.right:
        traversePostOrder(node.right, collector)
    collector.collect(node.data)

# build a sample tree
tree = Tree(Node(50), "Tree Traversals")
tree.root.left = Node(25)
tree.root.right = Node(75)
tree.root.left.left = Node(10)
tree.root.left.right = Node(35)
tree.root.left.right.left = Node(30)
tree.root.left.right.right = Node(42)
tree.root.left.left.left = Node(5)
tree.root.left.left.right = Node(13)

# print the tree as a visual cue
tree.print()

TraversalExecutor.execute(tree.root, traversePreOrder, "preOrder")
TraversalExecutor.execute(tree.root, traverseInOrder, "inOrder")
TraversalExecutor.execute(tree.root, traversePostOrder, "postOrder")

Tree Traversals 
              50* 
       /             \
      25              75              
   /     \         /     \
  10      35      _       _       
 / \     / \     / \     / \
5   13  30  42  _   _   _   _   

preOrder: 50, 25, 10, 5, 13, 35, 30, 42, 75
inOrder: 5, 10, 13, 25, 30, 35, 42, 50, 75
postOrder: 5, 13, 10, 30, 42, 35, 25, 75, 50


# Tree height

The tree height is important, as from this metric one can determine how many jumps will be needed (in the worst case) in order to reach an item.  
Let's see how to calculate the tree height.

In [13]:
from tree import Node
from tree import Tree

def height(node, h = 0):
    leftHeight = height(node.left, h + 1) if node.left else h
    rightHeight = height(node.right, h + 1) if node.right else h
    return max(leftHeight, rightHeight)

# build a sample tree
tree = Tree(Node(50), "Tree Traversals")
tree.root.left = Node(25)
tree.root.right = Node(75)
tree.root.left.left = Node(10)
tree.root.left.right = Node(35)
tree.root.left.right.left = Node(30)
tree.root.left.right.right = Node(42)
tree.root.left.left.left = Node(5)
tree.root.left.left.right = Node(13)

# print the tree as a visual cue
tree.print()

print("Tree height is {}".format(height(tree.root)))

Tree Traversals 
              50* 
       /             \
      25              75              
   /     \         /     \
  10      35      _       _       
 / \     / \     / \     / \
5   13  30  42  _   _   _   _   

Tree height is 3


# Enumerate nodes by depth

See how we can enumerate nodes at every depth of a tree.

In [56]:
from tree import Node
from tree import Tree

def height(node, h = 0):
    leftHeight = height(node.left, h + 1) if node.left else h
    rightHeight = height(node.right, h + 1) if node.right else h
    return max(leftHeight, rightHeight)

def __getNodesAtDepth(node, depth, nodes):
    if depth == 0:
        nodes.append(node)
        return nodes
    if node.left:
        __getNodesAtDepth(node.left, depth - 1, nodes)
    else:
        nodes.extend([None] * 2 ** (depth - 1))
    if node.right:
        __getNodesAtDepth(node.right, depth - 1, nodes)
    else:
        nodes.extend([None] * 2 ** (depth - 1))
    return nodes

def getNodesAtDepth(node, depth):
    return __getNodesAtDepth(node, depth, [])

def node_array_to_str(nodeArray):
    str_list = [str(node.data if node else None) for node in nodeArray]
    # create a row with values separated by commas
    return ", ".join(str_list)

# build a sample tree
tree = Tree(Node(50), "Tree nodes at individual depths")
tree.root.left = Node(25)
tree.root.right = Node(75)
tree.root.left.left = Node(10)
tree.root.left.right = Node(35)
tree.root.left.right.left = Node(30)
tree.root.left.right.right = Node(42)
tree.root.left.left.left = Node(5)
tree.root.left.left.right = Node(13)

# print the tree as a visual cue
tree.print()

tree_heigth = height(tree.root)

for i in range(0, tree_heigth + 1):
    print("Nodes at depth [{}]: {}".format(i, node_array_to_str(getNodesAtDepth(tree.root, i))))


Tree nodes at individual depths 
              50* 
       /             \
      25              75              
   /     \         /     \
  10      35      _       _       
 / \     / \     / \     / \
5   13  30  42  _   _   _   _   

Nodes at depth [0]: 50
Nodes at depth [1]: 25, 75
Nodes at depth [2]: 10, 35, None, None
Nodes at depth [3]: 5, 13, 30, 42, None, None, None, None
